# Part I Python Fundamentals 4

This is my review note of Python for the purpose of self-study. The note mixes up with English & Chinese.
- Part I follows the *[Liao's Python tutorial](https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000) (in Chinese)*

In [24]:
# Display multiple interactive objects in one shell
# No Need for print function
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### 8.错误、调试和测试

错误
- 可预测错误
- 不可预测错误（异常）： 比如写入文件的时候，磁盘满了，写不进去了，或者从网络抓取数据，网络突然断掉了

### try: 如果我们觉得要运行的代码可能出错的时候用

In [4]:
# 出现错误就会跳转到 except， 没有错误跳转到 else
# 然后执行 finally 如果有的话
try:
    print('try...')
    r = 10 / 0 # error
    print('result:', r) # 不会被执行，直接跳转到 except
except ZeroDivisionError as e:
    print('except:', e)
except ValueError as e: # 可以有多个 except 并列存在
    print('ValueError')
    
# 此外，如果没有错误发生，可以在except语句块后面加一个else，当没有错误发生时，会自动执行else语句：
else:
    print('no error!')

# 无论有没有错误 finally 都会被执行
finally:
    print('finally...')

try...
except: division by zero
finally...


### Python的错误其实也是class，所有的错误类型都继承自BaseException，所以在使用except时需要注意的是，它不但捕获该类型的错误，还把其子类也“一网打尽”
[python3 doc 所有 error 类型继承](https://docs.python.org/3/library/exceptions.html#exception-hierarchy)

### logging
- 记录错误，并且继续运行到底

In [6]:
import logging

try:
    x = 10 / 0 # 出错
except ZeroDivisionError as e:
    logging.exception(e) # 记录错误，继续运行
    
print('end')

# 通过配置，logging还可以把错误记录到日志文件里，方便事后排查。

end


另外还可以主动抛出错误用 `raise`

### 调试

In [7]:
# 1. print( ) 
# 缺点是还需要删掉

In [10]:
# 2. assert

def foo(s):
    n = int(s)
    assert n != 0, 'n is zero!' # 这个表达式必须是 True
    return 10 / n

foo('0')

AssertionError: n is zero!

- 启动Python解释器时可以用 -O 参数来关闭 assert
- python3 -O err.py
- 关闭后，你可以把所有的assert语句当成pass来看

In [17]:
# 3. logging
# print 换位 logging
# 跟 assert 相比， logging 会把错误输出到文件
import logging
logging.basicConfig(level=logging.INFO)

s = '0'
n = int(s)
logging.info('n = %d' % n)
print(10 / n)

# 用 python err.py 来运行

ZeroDivisionError: division by zero

### logging 有很多级别 debug, info, warning, error, critical 一个比一个严重
- 当 level=info 的时候，那么 debug 消息就不起作用了

In [19]:
# 4. -m pdb 一步步运行
# python3 -m pdb err.py

s = '0'
n = int(s)
print(10 / n)

# Terminal 上输入 python3 -m pdb err.py
# -> 箭头指向未运行语句

ZeroDivisionError: division by zero

如果代码太长 10000+ 行怎么办？

In [12]:
import pdb

s = '0'
n = int(s)
pdb.set_trace()# 运行到这里会自动暂停,进入pdb调试环境
print(10 / n)

# c continue 停止测试环境，继续运行

--Return--
> <ipython-input-12-69750b5f83cc>(5)<module>()->None
-> pdb.set_trace()# 运行到这里会自动暂停,进入pdb调试环境
(Pdb) q


BdbQuit: 

In [13]:
# 5. use the fucking IDE 

**虽然用IDE调试起来比较方便，但是最后你会发现，logging才是终极武器。 WTF??**

### 单元测试
- 单元测试是用来对一个模块、一个函数或者一个类来进行正确性检验的测试工作

单元测试通过后有什么意义呢？
- **源代码更新版本之后，只需要跑一边单元测试，就知道原来的功能是否都还能实现，而不需要重新手动测试。**

In [15]:
# 首先，我们来编写一个Dict类，这个类的行为和dict一致，但是可以通过属性来访问
# >>> d = Dict(a=1, b=2)
# >>> d['a']
# 1
# >>> d.a
# 1

def Dict(dict):
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs) # copy 了父类的初始化内容
        
    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError(r"'Dict' object has no attribute '%s'" % key)

    def __setattr__(self, key, value):
        self[key] = value

为了编写单元测试，我们需要引入Python自带的unittest模块

In [123]:
import unittest

from mydict import Dict

class TestDict(unittest.TestCase):

    # 以test开头的方法就是测试方法，不以test开头的方法不被认为是测试方法，测试的时候不会被执行
    def test_init(self):
        d = Dict(a=1, b='test')
        self.assertEqual(d.a, 1) # 断言函数返回的结果与1相等
        self.assertEqual(d.b, 'test')
        self.assertTrue(isinstance(d, dict))

    def test_key(self):
        d = Dict()
        d['key'] = 'value'
        self.assertEqual(d.key, 'value')

    def test_attr(self):
        d = Dict()
        d.key = 'value'
        self.assertTrue('key' in d)
        self.assertEqual(d['key'], 'value')

    def test_keyerror(self):
        d = Dict()
        with self.assertRaises(KeyError): # 断言会抛出KeyError
            value = d['empty']

    def test_attrerror(self):
        d = Dict()
        with self.assertRaises(AttributeError):
            value = d.empty

# 运行方法
# 1. 可以直接运行
if __name__ == '__main__':
    unittest.main()

# 2. 另一种方法是在命令行通过参数-m unittest直接运行单元测试：
# python3 -m unittest mydict_test
# 这是推荐的做法，因为这样可以一次批量运行很多单元测试

E
ERROR: /Users/zhengtingli/Library/Jupyter/runtime/kernel-ddde5b1c-1f42-4e97-8b48-9b7be581a314 (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute '/Users/zhengtingli/Library/Jupyter/runtime/kernel-ddde5b1c-1f42-4e97-8b48-9b7be581a314'

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (errors=1)


SystemExit: True

To exit: use 'exit', 'quit', or Ctrl-D.


### `super()` 子类里访问父类的同名属性/方法，而又不想直接引用父类的名字

In [18]:
class A:
    def m(self):
        print('A')

class B(A): 
    def m(self): # 同名 method
        super().m() # 引用父类同名 method
        print('B')

B().m()

A
B


用以下 unittest 来找出 student.py 的 bug

In [1]:
from student import Student
import unittest

class TestStudent(unittest.TestCase):

    def test_80_to_100(self):
        s1 = Student('Bart', 80)
        s2 = Student('Lisa', 100)
        self.assertEqual(s1.get_grade(), 'A')
        self.assertEqual(s2.get_grade(), 'A')

    def test_60_to_80(self):
        s1 = Student('Bart', 60)
        s2 = Student('Lisa', 79)
        self.assertEqual(s1.get_grade(), 'B')
        self.assertEqual(s2.get_grade(), 'B')

    def test_0_to_60(self):
        s1 = Student('Bart', 0)
        s2 = Student('Lisa', 59)
        self.assertEqual(s1.get_grade(), 'C')
        self.assertEqual(s2.get_grade(), 'C')

    def test_invalid(self):
        s1 = Student('Bart', -1)
        s2 = Student('Lisa', 101)
        with self.assertRaises(ValueError):
            s1.get_grade()
        with self.assertRaises(ValueError):
            s2.get_grade()

if __name__ == '__main__':
    unittest.main()

E
ERROR: /Users/zhengtingli/Library/Jupyter/runtime/kernel-ddde5b1c-1f42-4e97-8b48-9b7be581a314 (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute '/Users/zhengtingli/Library/Jupyter/runtime/kernel-ddde5b1c-1f42-4e97-8b48-9b7be581a314'

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (errors=1)


SystemExit: True

To exit: use 'exit', 'quit', or Ctrl-D.


unittest 真的非常好用！

### 文档测试 doctest
- 当我们编写注释时，如果写上这样的注释：

In [39]:
def abs(n):
    '''
    Function to get absolute value of number.

    Example:

    >>> abs(1)
    1
    >>> abs(-1)
    1
    >>> abs(0)
    0
    >>> abs('a')
    Traceback (most recent call last):
    ... 
    TypeError: unorderable types: str() >= int()
    '''
    return n if n >= 0 else (-n)

**测试异常的时候，可以用...表示中间一大段烦人的输出**

Python内置的“文档测试”（doctest）模块可以直接提取注释中的代码并执行测试

In [40]:
import doctest
doctest.testmod()

TestResults(failed=0, attempted=4)

总结

- 测试写在 doc 里面，我们就直接进行 doctest
- 测试写成单独的 unittest.py, 我们就直接运行 unittest

### 9.同步I/O Programming

由于CPU和内存的速度远远高于外设的速度，所以，在IO编程中，就存在速度严重不匹配的问题。举个例子来说，比如要把100M的数据写入磁盘，CPU输出100M的数据只需要0.01秒，可是磁盘要接收这100M数据可能需要10秒，怎么办呢？有两种办法：

第一种是CPU等着，也就是程序暂停执行后续代码，等100M的数据在10秒后写入磁盘，再接着往下执行，这种模式称为同步IO；

另一种方法是CPU不等待，只是告诉磁盘，“您老慢慢写，不着急，我接着干别的事去了”，于是，后续代码可以立刻接着执行，这种模式称为异步IO。

读文件

In [ ]:
# 普通读取 utf-8
with open('/path/to/file', 'r') as f:
    print(f.read())
    
# with 省去了 try ... finally .. f.close() 的套路

# 读取大文件
# 1. read(size) size 个 characters
# 2. readline
for line in f.readlines():
    print(line.strip()) # 把末尾的'\n'删掉

In [ ]:
# 二进制读取 用 'rb'
f = open('/Users/michael/test.jpg', 'rb') 

In [ ]:
# 读取非UTF-8编码 加入 encoding 参数在 open() 中
f = open('/Users/michael/gbk.txt', 'r', encoding='gbk', errors='ignore')
# errors='ignore' 忽略文件中的非法编码

写文件

In [ ]:
# ‘w' 普通写入
# 'wb' 二进制写入
# 写入特定编码的文本文件， 加入 encoding 参数在 open() 中
with open('/Users/michael/test.txt', 'w') as f:
    f.write('Hello, world!')

StringIO/BytesIO

Memory 中读写 str / bytes, 而不是在文件读写

In [21]:
from io import StringIO, BytesIO

f = StringIO() # 要有括号！
f.write('hi') # write into the memory, then return len(str)
f.write('iam')
f.write('Mike')
f.getvalue() # get the whole str

b = BytesIO()
b.write('我的python 666'.encode('utf-8'))

16

In [53]:
# 也可以初始化一个长 str 进行读取
f = StringIO('Hello!\nHi!\nGoodbye!')
while True:
    line = f.readline()
    if line =='':
        break
    print(line.strip())

Hello!
Hi!
Goodbye!


操作文件和目录

In [25]:
# 环境变量
import os
os.environ
os.environ.get('PATH') # 获取某个环境变量的值

environ({'SECURITYSESSIONID': '186a6', 'TERM_PROGRAM_VERSION': '343.7', 'LOGNAME': 'zhengtingli', 'PAGER': 'cat', 'PWD': '/Users/zhengtingli/Desktop/fluent_python', 'JPY_PARENT_PID': '20825', 'SHLVL': '1', 'TERM_PROGRAM': 'Apple_Terminal', 'USER': 'zhengtingli', 'TERM': 'xterm-color', 'TERM_SESSION_ID': '079C3D17-A752-4970-BC10-8D7F70812E64', '__CF_USER_TEXT_ENCODING': '0x1F5:0x0:0x0', 'SHELL': '/bin/bash', 'SSH_AUTH_SOCK': '/private/tmp/com.apple.launchd.iFIlwn2rIC/Listeners', 'XPC_FLAGS': '0x0', 'OLDPWD': '/Users/zhengtingli/Desktop', '_': '/Users/zhengtingli/anaconda/bin/jupyter', 'HOME': '/Users/zhengtingli', 'CLICOLOR': '1', 'LANG': 'en_US.UTF-8', 'Apple_PubSub_Socket_Render': '/private/tmp/com.apple.launchd.so1hwWRQh3/Render', 'XPC_SERVICE_NAME': '0', 'PATH': '/Users/zhengtingli/anaconda/bin:/Library/Frameworks/Python.framework/Versions/3.5/bin:/Library/Frameworks/Python.framework/Versions/3.4/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/TeX/texbin', 'GIT_PAGER': 'ca

'/Users/zhengtingli/anaconda/bin:/Library/Frameworks/Python.framework/Versions/3.5/bin:/Library/Frameworks/Python.framework/Versions/3.4/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/TeX/texbin'

In [68]:
# 查看 cwd 的绝对路径:
os.path.abspath('.')

# 在某个目录下创建一个新目录(合并路径）:
os.path.join('some_directory', 'new_direcotry')
# 创建了 '/some_directory/new_directory'

# 拆分路径
os.path.split('/Users/michael/testdir/file.txt')
# ('/Users/michael/testdir', 'file.txt')

# 得到文件扩展名
os.path.splitext('/path/file.txt')
# ('/path/file', '.txt')

# 这些合并、拆分路径的函数并不要求目录和文件要真实存在，它们只对字符串进行操作。

os.mkdir 创建
os.rmdir 删除
os.rename('old.txt', 'new.txt')
os.remove('new.txt')
os.listdir() 列出所有所有的文件

'/Users/zhengtingli/Desktop/fluent_python'

In [71]:
# 列出所有的.py文件
# Fucking cool
[x for x in os.listdir('.') if os.path.isfile(x) and os.path.splitext(x)[1] == '.py']

['mydict.py', 'student.py']

In [72]:
# 列出当前目录下的所有 directory
[x for x in os.listdir('.') if os.path.isdir(x)]

['.git', '.ipynb_checkpoints', '__pycache__', 'books', 'img']

In [96]:
# ? 编写一个程序，能在当前目录以及当前目录的所有子目录下查找文件名包含指定字符串'py'的文件，并打印出相对路径
import os

def print_file(directory):
    for f in os.listdir(directory):
        file_path = os.path.join(directory, f)
        if os.path.isfile(f) and 'py' in f:
            print(os.path.abspath(f))
        elif os.path.isdir(f):
            print_file(file_path)
        else:
            pass

print_file('.')
# 为什么不好用啊啊啊啊啊？

/Users/zhengtingli/Desktop/fluent_python/Ipython_notebook_useful_features
/Users/zhengtingli/Desktop/fluent_python/mydict.py
/Users/zhengtingli/Desktop/fluent_python/Part I Python Fundamentals 1.ipynb
/Users/zhengtingli/Desktop/fluent_python/Part I Python Fundamentals 2.ipynb
/Users/zhengtingli/Desktop/fluent_python/Part I Python Fundamentals 3.ipynb
/Users/zhengtingli/Desktop/fluent_python/Part I Python Fundamentals 4.ipynb
/Users/zhengtingli/Desktop/fluent_python/Part II Fluent Python 1.ipynb
/Users/zhengtingli/Desktop/fluent_python/student.py


In [124]:
# 找问题，没解决
for f in os.listdir('./books'):
    print(f)
    if os.path.isfile(f):
        print(f)
    elif os.path.isabs(f):
        print('a')
    elif os.path.ismount(f):
        print('m')
    elif os.path.islink(f):
        print('l')
    else:
        print('unrecognized') # fuck why??

.DS_Store
.DS_Store
fluent-python.pdf
unrecognized
流畅的python_笔记版.pdf
unrecognized


### Pickling(二进制储存）

Pickling
我们把变量从内存中变成可存储或传输的过程称之为序列化，在Python中叫pickling 序列化之后，就可以把序列化后的内容写入磁盘，或者通过网络传输到别的机器上

In [106]:
import pickle
lst = [1, 2, 3]
pickle.dumps(lst) # dumps 倒出来

b'\x80\x03]q\x00(K\x01K\x02K\x03e.'

In [107]:
# dump
with open('dump.txt', 'wb') as f: # 创建一个文件
    pickle.dump(lst, f) # dump 到该文件里

In [108]:
# load
with open('dump.txt', 'rb') as f:
    pickle.load(f)

[1, 2, 3]

### Json(string储存)

如果我们要在不同的编程语言之间传递对象，就必须把对象序列化为标准格式 JSON (用 string 格式来储存) (utf - 8)
- JSON表示的对象就是标准的JavaScript语言的对象，JSON和Python内置的数据类型对应如下：

In [27]:
import json
lst = [1, 2, 3]
json.dumps(lst) # 可见储存格式是 string

'[1, 2, 3]'

In [113]:
import json
lst = [1, 2, 3]
with open('dump_json.txt', 'w') as f: # 'wb' -> 'w'
    json.dump(lst, f)

In [115]:
with open('dump_json.txt', 'r') as f:
    json.load(f)

[1, 2, 3]

In [121]:
import json
obj = dict(name='小明', age=20)
s = json.dumps(obj)

json 有很多[可选参数](https://docs.python.org/3/library/json.html#json.dumps)